In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [1]:
import gzip
import pickle
from utils.util import *
from train_test import *
import sys
import os
import argparse
from parse_config import ConfigParser

In [2]:
def load_compressed_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [3]:
restored_data = load_compressed_pickle('/datasets/mind_data/data_dict_compressed.pickle')

In [4]:
user_history=restored_data["user_history"]
entity_embedding=restored_data["entity_embedding"]
relation_embedding=restored_data["relation_embedding"]
entity_adj=restored_data["entity_adj"]
relation_adj=restored_data["relation_adj"]
news_feature=restored_data["news_feature"]
max_entity_freq=restored_data["max_entity_freq"]
max_entity_pos=restored_data["max_entity_pos"]
max_entity_type=restored_data["max_entity_type"]
train_data=restored_data["train_data"]
dev_data=restored_data["dev_data"]
vert_train=restored_data["vert_train"]
vert_test=restored_data["vert_test"]
pop_train=restored_data["pop_train"]
pop_test=restored_data["pop_test"]
item2item_train=restored_data["item2item_train"]
item2item_test=restored_data["item2item_test"]

In [5]:
parser = argparse.ArgumentParser(description='KRED')
parser.add_argument('-f')
parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')
config = ConfigParser.from_args(parser)

In [6]:
epochs = 5
batch_size = 64
train_type = "single_task"
task = "user2item"

config['trainer']['epochs'] = epochs
config['data_loader']['batch_size'] = batch_size
config['trainer']['training_type'] = train_type
config['trainer']['task'] = task

In [7]:
if config['trainer']['training_type']  == "multi-task":
    data = user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, train_data, dev_data, vert_train, vert_test, pop_train, pop_test, item2item_train, item2item_test
elif config['trainer']['task'] == "user2item":
    data = user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, train_data, dev_data
elif config['trainer']['task'] == "item2item":
    data =  user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, item2item_train, item2item_test
elif config['trainer']['task'] == "vert_classify":
    data = user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, vert_train, vert_test
elif config['trainer']['task'] == "pop_predict":
    data = user_history, entity_embedding, relation_embedding, entity_adj, relation_adj, news_feature, max_entity_freq, max_entity_pos, max_entity_type, pop_train, pop_test

In [8]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [9]:
#Single Task Training

In [10]:
user_history_dict, entity_embedding, relation_embedding, entity_adj, relation_adj, doc_feature_dict, entity_num, position_num, type_num, train_data, test_data = data

In [11]:
train_data_u2i = NewsDataset(train_data)

In [12]:
train_sampler_u2i = RandomSampler(train_data_u2i)

In [13]:
(train_data['item2'][0])

['N35729', 'N0', 'N0', 'N0', 'N55689']

In [14]:
train_dataloader_u2i = DataLoader(train_data_u2i, sampler=train_sampler_u2i,
                                          batch_size=config['data_loader']['batch_size'],
                                          collate_fn=my_collate_fn, pin_memory=False)

In [15]:
criterion = Softmax_BCELoss(config)

In [16]:
train_data_loader = train_dataloader_u2i

In [17]:
device, deviceids = prepare_device(config['n_gpu'])

In [18]:
deviceids

[0]

In [19]:
model = KREDModel(config, user_history_dict, doc_feature_dict, entity_embedding, relation_embedding, entity_adj,relation_adj, entity_num, position_num, type_num).cuda()

In [20]:
optimizer = optim.Adam(model.parameters(), lr=config['optimizer']['lr'], weight_decay=0)

In [21]:
trainer = Trainer(config, model, criterion, optimizer, device, train_data_loader, data[-1])

In [ ]:
trainer.train()

model training
Training epoch 0/6 - 0.0
######
 Step: 0, 0.0 
######


/notebooks/model/News_embedding.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  entity_num_embedding = self.entity_num_embeddings(torch.tensor(entity_nums).cuda())
/notebooks/model/News_embedding.py:155: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  aggregate_embedding, topk_index = self.attention_layer(news_entity_embedding, torch.FloatTensor(context_vecs).cuda())


######
 Step: 100, 0.027078256160303276 
######
######
 Step: 200, 0.05415651232060655 
######
######
 Step: 300, 0.08123476848090982 
######
######
 Step: 400, 0.1083130246412131 
######
######
 Step: 500, 0.13539128080151638 
######
######
 Step: 600, 0.16246953696181965 
######
######
 Step: 700, 0.18954779312212294 
######
######
 Step: 800, 0.2166260492824262 
######
######
 Step: 900, 0.2437043054427295 
######
######
 Step: 1000, 0.27078256160303277 
######
######
 Step: 1100, 0.29786081776333606 
######
######
 Step: 1200, 0.3249390739236393 
######
######
 Step: 1300, 0.3520173300839426 
######
######
 Step: 1400, 0.3790955862442459 
######
######
 Step: 1500, 0.4061738424045491 
######
######
 Step: 1600, 0.4332520985648524 
######
######
 Step: 1700, 0.4603303547251557 
######
######
 Step: 1800, 0.487408610885459 
######
######
 Step: 1900, 0.5144868670457623 
######
######
 Step: 2000, 0.5415651232060655 
######
######
 Step: 2100, 0.5686433793663688 
######
######
 Step: 

In [ ]:
from model.News_embedding import News_embedding

In [ ]:
news_232 = News_embedding(config, doc_feature_dict, entity_embedding, relation_embedding, entity_adj, relation_adj, entity_num, position_num, type_num)

In [ ]:
news_id = [['N56613', 'N56613', 'N548', 'N12889', 'N2994', 'N19829', 'N5123', 'N3346', 'N56586', 'N2665', 'N51061', 'N23817', 'N50454', 'N8391', 'N21623', 'N43295', 'N45589']]
#'N56613', 'N548', 'N12889', 'N2994', 'N19829', 'N5123', 'N3346', 'N56586', 'N2665', 'N51061', 'N23817', 'N50454', 'N8391', 'N21623', 'N43295', 'N45589'

In [ ]:
entity_ids = news_232.get_entities_ids(news_id)

In [ ]:
kgat2232 = news_232.kgat

In [ ]:
entity_ids = kgat2232.entity_ids_clearner(entity_ids)

In [ ]:
neighbor_entities, neighbor_relations = kgat2232.get_neighbors(entity_ids)

In [ ]:
entity_embedding_lookup = nn.Embedding.from_pretrained(kgat2232.entity_embedding)
relation_embedding_lookup = nn.Embedding.from_pretrained(kgat2232.relation_embedding)
neighbor_entity_embedding = entity_embedding_lookup(torch.tensor(neighbor_entities))
neighbor_relation_embedding = relation_embedding_lookup(torch.tensor(neighbor_relations))
entity_embedding = entity_embedding_lookup(torch.tensor(entity_ids))

In [ ]:
entity_embedding_expand = torch.unsqueeze(entity_embedding, 3)
entity_embedding_expand = entity_embedding_expand.expand(entity_embedding_expand.shape[0],
                                                         entity_embedding_expand.shape[1],
                                                         entity_embedding_expand.shape[2],
                                                         kgat2232.config['model']['entity_neighbor_num'],
                                                         entity_embedding_expand.shape[4])
embedding_concat = torch.cat(
    [entity_embedding_expand, neighbor_entity_embedding, neighbor_relation_embedding], 4)
embedding_concat = embedding_concat.to(torch.float32)
embedding_concat

In [ ]:
attention_value = kgat2232.softmax(kgat2232.attention_layer2(kgat2232.relu(kgat2232.attention_layer1(embedding_concat_))))
neighbor_att_embedding = torch.sum(attention_value * neighbor_entity_embedding, dim=3)

In [ ]:
kgat_embedding = kgat2232.aggregate(entity_embedding, neighbor_att_embedding)

In [ ]:
kgat_embedding

In [ ]:
embedding_concat.get_device()

In [ ]:
entity_embedding_expand = entity_embedding_expand.expand(entity_embedding_expand.shape[0], entity_embedding_expand.shape[1],entity_embedding_expand.shape[2], config['model']['entity_neighbor_num'], entity_embedding_expand.shape[4])

In [ ]:
embedding_concat = torch.cat([entity_embedding_expand, neighbor_entity_embedding, neighbor_relation_embedding], 4).cuda()

In [ ]:
attention_value =  kgat2232.softmax(kgat2232.attention_layer2(kgat2232.relu(kgat2232.attention_layer1(embedding_concat))))

In [ ]:
neighbor_entities_tensor = torch.tensor(neighbor_entities)
filter_mask = torch.tensor(neighbor_entities) < 3241390
neighbor_entities_tensor[~filter_mask] = 0

In [ ]:
entity_embedding = entity_embedding_lookup(entity_ids_tensor)



In [ ]:
entity_ids_tensor = torch.tensor(entities).cuda()
filter_mask = (entity_ids_tensor < 3241390).cuda()
entity_ids_tensor[~filter_mask] = 0

In [ ]:
neighbor_entity_embedding = entity_embedding_lookup(neighbor_entities_tensor)

In [ ]:
neighbor_entity_embedding.shape

In [ ]:
entities = [[[172, 0]]]

In [ ]:
def get_neighbors(entities):
    neighbor_entities = []
    neighbor_relations = []
    for entity_batch in entities:
        neighbor_entities.append([])
        neighbor_relations.append([])
        for entity in entity_batch:
            if type(entity) == int:
                neighbor_entities[-1].append(entity_adj_2[entity])
                neighbor_relations[-1].append(relation_adj_2[entity])
            else:
                neighbor_entities[-1].append([])
                neighbor_relations[-1].append([])
                for entity_i in entity:
                    neighbor_entities[-1][-1].append(entity_adj_2[entity_i])
                    neighbor_relations[-1][-1].append(relation_adj_2[entity_i])

    return neighbor_entities, neighbor_relations

tmp_nei = get_neighbors(entities)
used_nei = kgat2232.get_neighbors(entities)


In [ ]:
used_nei[0]

In [ ]:
for x in tmp_nei[0][0][0][0]:
    if x in used_nei[0][0][0][0]:
        print(f'ok {x}')
    else:
        print(f'x {x}')


In [ ]:
kgat2232.entity_embedding.shape

In [ ]:
used_nei[0]

In [ ]:
def get_entity_embedding(neighbor_entities):
    entity_embedding_batch = []
    if type(neighbor_entities[0][0]) == int:
        neighbor_entities = torch.LongTensor(neighbor_entities)
        for i in range(len(neighbor_entities)):
            entity_embedding_batch.append([])
            for j in range(len(neighbor_entities[i])):
                entity_embedding_batch[i].append([])
                for entityid in neighbor_entities[i][j]:
                    entity_embedding_batch[i][j].append(self.entity_embedding[entityid])
    else:
        neighbor_entities = torch.LongTensor(neighbor_entities)
        for i in range(len(neighbor_entities)):
            entity_embedding_batch.append([])
            for j in range(len(neighbor_entities[i])):
                entity_embedding_batch[i].append([])
                for k in range(len(neighbor_entities[i][j])):
                    entity_embedding_batch[i][j].append([])
                    for entityid in neighbor_entities[i][j][k]:
                        entity_embedding_batch[i][j][k].append(self.entity_embedding[entityid])
    return torch.FloatTensor(torch.stack(entity_embedding_batch)).cuda()

In [ ]:
entity_embedding_lookup(torch.tensor(neighbor_entities))

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
kgat2232.entity_embedding

In [ ]:
torch.clone(entity_embedding).cuda()

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
torch.tensor(kgat2232.entity_embedding).cuda()

In [ ]:
entity_embedding_lookup = nn.Embedding.from_pretrained(kgat2232.entity_embedding)
relation_embedding_lookup = nn.Embedding.from_pretrained(kgat2232.relation_embedding)

In [ ]:
neighbor_relation_embedding = relation_embedding_lookup(torch.tensor(neighbor_relations))

In [ ]:
entity_embedding = entity_embedding_lookup(torch.tensor(entities))

In [ ]:
print(f'{len(entity_nums)}, {len(entity_nums[0])}, {len(entity_nums[0][0])}')


In [ ]:
entity_num_embedding = news_232.get_entity_num_embedding(entity_nums)


In [ ]:
kgat2232.entity_embedding.shape

In [ ]:
torch.min(kgat2232.entity_embedding)

In [ ]:
torch.nan_to_num(kgat2232.entity_embedding)

In [ ]:
kgat2232.entity_embedding[:2,:].cuda()

In [ ]:
import torch
torch.zeros([3000000000,100]).cuda()

In [ ]:
!nvidia-smi

In [ ]:
entity_adj

In [ ]:
len([['N20076', 'N45590', 'N13579', 'N47817', 'N17583'], ['N45375', 'N52446', 'N62416', 'N45543', 'N15952'], ['N24686', 'N61006', 'N12905', 'N45076', 'N63273'], ['N59981', 'N13801', 'N32791', 'N8957', 'N154'], ['N3418', 'N14029', 'N976', 'N33831', 'N56214'], ['N13930', 'N55204', 'N19542', 'N43502', 'N38662'], ['N50872', 'N0', 'N0', 'N0', 'N35815'], ['N20069', 'N32544', 'N41020', 'N33619', 'N63970'], ['N55645', 'N44324', 'N41881', 'N3491', 'N3894'], ['N50872', 'N6477', 'N51570', 'N36226', 'N41387'], ['N8373', 'N8595', 'N41934', 'N6837', 'N29212'], ['N57651', 'N36681', 'N0', 'N0', 'N49685'], ['N21753', 'N28495', 'N63970', 'N0', 'N58363'], ['N31763', 'N28413', 'N4912', 'N45076', 'N50314'], ['N646', 'N62360', 'N30475', 'N58363', 'N50675'], ['N3418', 'N18423', 'N26224', 'N40468', 'N19444'], ['N3344', 'N60858', 'N5364', 'N19592', 'N38159'], ['N20041', 'N64748', 'N45016', 'N60105', 'N63154'], ['N35737', 'N63550', 'N55689', 'N55204', 'N4936'], ['N16148', 'N42457', 'N56214', 'N10746', 'N21712'], ['N20041', 'N30475', 'N30920', 'N34956', 'N6379'], ['N38215', 'N4642', 'N13907', 'N39073', 'N46279'], ['N8559', 'N31448', 'N62049', 'N39187', 'N21712'], ['N51478', 'N21741', 'N2350', 'N21509', 'N54489'], ['N37088', 'N41178', 'N39115', 'N59852', 'N24272'], ['N53111', 'N13051', 'N30108', 'N61214', 'N61572'], ['N60992', 'N59385', 'N20076', 'N25444', 'N14523'], ['N62386', 'N33677', 'N33885', 'N45704', 'N1940'], ['N20076', 'N42977', 'N48063', 'N14592', 'N28213'], ['N51255', 'N31978', 'N47817', 'N20678', 'N55606'], ['N37088', 'N16209', 'N58075', 'N18870', 'N32846'], ['N41578', 'N47981', 'N57090', 'N29128', 'N35729'], ['N44616', 'N42028', 'N61074', 'N18708', 'N1034'], ['N50058', 'N38442', 'N59267', 'N46739', 'N1539'], ['N36681', 'N41224', 'N29128', 'N57651', 'N50060'], ['N25722', 'N19318', 'N60577', 'N8349', 'N31978'], ['N49180', 'N41020', 'N3344', 'N20041', 'N58363'], ['N34579', 'N24423', 'N12029', 'N34915', 'N33240'], ['N61623', 'N2823', 'N42961', 'N33964', 'N46370'], ['N4912', 'N15855', 'N49685', 'N64228', 'N53017'], ['N11830', 'N42961', 'N20240', 'N45076', 'N112'], ['N32437', 'N35729', 'N13801', 'N61214', 'N55689'], ['N42803', 'N31679', 'N55943', 'N42860', 'N23446'], ['N59407', 'N35576', 'N27737', 'N54752', 'N62318'], ['N42416', 'N25138', 'N61623', 'N20224', 'N55689'], ['N48017', 'N50872', 'N11817', 'N50592', 'N38779'], ['N22417', 'N34048', 'N35958', 'N4642', 'N36261'], ['N51896', 'N40065', 'N16342', 'N51346', 'N23414'], ['N34600', 'N27737', 'N13056', 'N42670', 'N57733'], ['N26977', 'N349', 'N55645', 'N24272', 'N13423'], ['N55355', 'N52294', 'N28983', 'N154', 'N3841'], ['N57733', 'N60550', 'N47576', 'N61429', 'N36186'], ['N57651', 'N36681', 'N0', 'N0', 'N49685'], ['N33619', 'N2952', 'N40356', 'N3737', 'N7328'], ['N19318', 'N646', 'N50675', 'N1421', 'N35133'], ['N6926', 'N60374', 'N47020', 'N39342', 'N1034'], ['N47061', 'N56214', 'N8509', 'N21420', 'N7618'], ['N30518', 'N46029', 'N46526', 'N41222', 'N53585'], ['N4642', 'N64252', 'N14029', 'N51048', 'N56214'], ['N22257', 'N55275', 'N15574', 'N19019', 'N6816'], ['N7821', 'N57283', 'N26488', 'N34110', 'N64902'], ['N33240', 'N29822', 'N37905', 'N11087', 'N7821'], ['N35773', 'N34504', 'N19050', 'N39949', 'N61949'], ['N61768', 'N61022', 'N54595', 'N41220', 'N1034']]
 )